In [ ]:

# 人脸检测模型导出
import paddlehub as hub

# 加载模型
face_detector = hub.Module(name="pyramidbox_lite_server")

# 导出推理模型
face_detector.save_inference_model(
    dirname='inference/face_detection',
    model_filename='__model__',
    params_filename='__params__')

In [ ]:
from paddlehub import Trainer

# 新建临时train.txt
#with open('/home/rogerle/PycharmProjects/WonkerAIMan/data/waves/train_list.txt', 'w', encoding='UTF-8') as f:
#    for x in range(37):
#        f.write('{:04d}.jpg 0\n'.format(x+1))

import os
from typing import Callable, Tuple

import paddle
import numpy as np
import paddlehub as hub
import paddlehub.vision.transforms as T
from paddlehub.finetune import trainer

#定义数据预处理方式
transforms = T.Compose([T.Resize((256, 256)),
                        T.CenterCrop(224),
                        T.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])],
                        to_rgb=True)
# 自定义数据集
class FaceDataset(paddle.io.Dataset):
    def __init__(self, transforms: Callable, mode: str = 'train'):
        # 数据集存放位置
        self.mode = mode
        self.transforms = transforms
        self.dataset_dir = "/home/rogerle/PycharmProjects/WonkerAIMan/data/waves"
        self.label_list = ['0','1']
        if self.mode == 'train':
            self.file = 'train_list.txt'
        elif self.mode == 'test':
            self.file = 'test_list.txt'
        else:
            self.file = 'validate_list.txt'

        self.file = os.path.join(self.dataset_dir, self.file)

        with open(self.file, 'r') as file:
            self.data = file.read().split('\n')

    def __getitem__(self, idx) -> Tuple[np.ndarray, int]:
        img_name, grt = self.data[idx].split(' ')
        img_path = os.path.join(self.dataset_dir, img_name)
        im = self.transforms(img_path)
        return im, int(grt)

    def __len__(self):
        return len(self.data)

dataset = FaceDataset(transforms, mode='train')
dataset_validate = FaceDataset(transforms,mode='val')

# 使用resnet50_vd_imagenet_ssld预训练模型进行finetune
model = hub.Module(name="resnet50_vd_imagenet_ssld",label_list=["0","1"])

optimizer = paddle.optimizer.Adam(learning_rate=0.001,parameters=model.parameters())
trainer = Trainer(model, optimizer, checkpoint_dir='./img_classification_ckpt',use_gpu=True,use_vdl=True)

trainer.train(dataset, epochs=100, batch_size=12, eval_dataset=dataset_validate, save_interval=10,log_interval=1)



In [1]:
import paddle
import paddlehub as hub
import numpy as np
import paddlehub.vision.transforms as T
from typing import Callable, Tuple
import os

model = hub.Module(name='resnet50_vd_imagenet_ssld', label_list=["0", "1"] ,load_checkpoint='/home/rogerle/PycharmProjects/WonkerAIMan/local/utils/wav2lip/img_classification_ckpt/best_model/model.pdparams')
result = model.predict(['0001.jpg'])
print (result)

/home/rogerle/PycharmProjects/WonkerAIMan/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
W0405 21:58:27.891147 18940 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 8.6, Driver API Version: 12.1, Runtime API Version: 11.7
W0405 21:58:27.891680 18940 gpu_resources.cc:91] device: 0, cuDNN Version: 8.8.


load custom parameters success
[{'0': 0.997945}]
